In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib, os

In [2]:
# ✅ Load dataset
wine = pd.read_csv("winequality-red-selected-missing.csv")
print("Dataset shape:", wine.shape)
wine.head()

In [3]:
# ✅ KNN Imputation for missing values
imputer = KNNImputer(n_neighbors=5)
wine_imputed = pd.DataFrame(imputer.fit_transform(wine), columns=wine.columns)

# ✅ Add binary target
wine_imputed["good_quality"] = (wine_imputed["quality"] >= 7).astype(int)
wine_imputed.head()

In [4]:
# ✅ Features & target
X = wine_imputed.drop(columns=["quality", "good_quality"])
y = wine_imputed["good_quality"]

# ✅ Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

In [5]:
# ✅ Train RandomForest model
model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=12)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [6]:
# ✅ Save model + scaler in artifacts/ so app.py can load them
os.makedirs("artifacts", exist_ok=True)
joblib.dump(model, "artifacts/wine_quality_model.pkl")
joblib.dump(scaler, "artifacts/scaler.pkl")
print("✅ Model and scaler saved in artifacts/ folder")